In [3]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

In [4]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [5]:
# def extract_linkedin_job_data():

#     headers = {
#         'x-rapidapi-key': rapidapi_key,
#         'x-rapidapi-host': rapidapi_host
#     }

#     location = "Australia"
#     limit = 100
#     offset = 0
#     titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

#     df_daily_all = pd.DataFrame()

#     for title_filter in titles:

#         #URL code the title filter and location filter
#         title_encoded=urllib.parse.quote(title_filter)
#         location_encoded=urllib.parse.quote(location)

#         #API endpoint 7day data query
#         #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

#         #API endpoint last 24 hour data query
#         base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
#         url =  f"https://{rapidapi_host}{base_url}"


#         response = requests.get(url, headers=headers)

#         if response.status_code == 200:
#             data = response.json()
#             # Convert the JSON data to a DataFrame
#             df_daily = json_normalize(data)
#             df_daily['job_category'] = title_filter
#             print(title_filter, df_daily.shape)
#             df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


#         else:
#             print(f"Error: {response.status_code}")

#     return df_daily_all

# df_daily_all = extract_linkedin_job_data()
# df_daily_all

In [ ]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1613031134,2025-05-02T05:33:16,2025-05-02T05:36:55.492225,Data Engineer,Data Engineer,NCS Group Australia,https://www.linkedin.com/company/ncs-group-aus...,2025-06-01T05:32:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, New South Wales",Privately Held,2000.0,"['IT Services', 'Technology Services', 'Artifi...","['Level 4, 10 Shelley Street, Sydney, New Sout...",NCS in Australia is reinventing technology ser...,False,Mid-Senior level,True,ncs-group-australia
1,1613035712,2025-05-02T05:21:43,2025-05-02T05:38:53.83865,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T05:21:43,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
2,1613025785,2025-05-02T05:21:43,2025-05-02T05:34:01.851283,Data Centre Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T05:21:43,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
3,1612950483,2025-05-02T04:41:08,2025-05-02T05:05:57.213031,AWS Data Engineer,Data Engineer,BlueRose Technologies,https://www.linkedin.com/company/bluerosetechn...,2025-06-01T04:41:07,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Bangalore, Karnataka",Privately Held,2010.0,"['PEGA', 'Telecom Billing', 'Siebel CRM', 'SAP...","['MS Square, 34/1-1, Langford Town, Bhemanna G...",BlueRose Technologies is a leading global prov...,False,Associate,True,bluerosetechnologies
4,1612910793,2025-05-02T04:36:12,2025-05-02T04:54:13.638235,Staff Data Engineer - Data Intelligence - Real...,Data Engineer,Culture Amp,https://www.linkedin.com/company/cultureamp,2025-06-01T04:36:12,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Richmond, Victoria",Privately Held,2010.0,"['Employee Engagement', 'Engagement Surveys', ...","['29 Stewart St, Level 2, Richmond, Victoria 3...",Culture Amp is the world’s leading employee ex...,False,Mid-Senior level,False,cultureamp


In [9]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['TITLE'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(37, 44)

In [10]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [11]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(37, 44)

In [13]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1613031134,2025-05-02T05:33:16,2025-05-02T05:36:55.492225,Data Engineer,Data Engineer,NCS Group Australia,https://www.linkedin.com/company/ncs-group-aus...,2025-06-01T05:32:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, New South Wales",Privately Held,2000.0,"['IT Services', 'Technology Services', 'Artifi...","['Level 4, 10 Shelley Street, Sydney, New Sout...",NCS in Australia is reinventing technology ser...,False,Mid-Senior level,True,ncs-group-australia
1,1613035712,2025-05-02T05:21:43,2025-05-02T05:38:53.83865,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T05:21:43,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank


### Process the df_daily_all data 


In [14]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-02
1    2025-05-02
2    2025-05-02
3    2025-05-02
4    2025-05-02
Name: job_date, dtype: object

In [15]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1613031134,2025-05-02T05:33:16,2025-05-02T05:36:55.492225,Data Engineer,Data Engineer,NCS Group Australia,https://www.linkedin.com/company/ncs-group-aus...,2025-06-01T05:32:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['IT Services', 'Technology Services', 'Artifi...","['Level 4, 10 Shelley Street, Sydney, New Sout...",NCS in Australia is reinventing technology ser...,False,Mid-Senior level,True,ncs-group-australia,2025-05-02,Melbourne,VIC
1,1613035712,2025-05-02T05:21:43,2025-05-02T05:38:53.83865,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T05:21:43,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-02,Woden,None
2,1612950483,2025-05-02T04:41:08,2025-05-02T05:05:57.213031,AWS Data Engineer,Data Engineer,BlueRose Technologies,https://www.linkedin.com/company/bluerosetechn...,2025-06-01T04:41:07,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['PEGA', 'Telecom Billing', 'Siebel CRM', 'SAP...","['MS Square, 34/1-1, Langford Town, Bhemanna G...",BlueRose Technologies is a leading global prov...,False,Associate,True,bluerosetechnologies,2025-05-02,Sydney,NSW
3,1612910793,2025-05-02T04:36:12,2025-05-02T04:54:13.638235,Staff Data Engineer - Data Intelligence - Real...,Data Engineer,Culture Amp,https://www.linkedin.com/company/cultureamp,2025-06-01T04:36:12,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Employee Engagement', 'Engagement Surveys', ...","['29 Stewart St, Level 2, Richmond, Victoria 3...",Culture Amp is the world’s leading employee ex...,False,Mid-Senior level,False,cultureamp,2025-05-02,Sydney,NSW
4,1612905307,2025-05-02T04:36:12,2025-05-02T04:52:31.787859,Staff Data Engineer - Data Intelligence - Real...,Data Engineer,Culture Amp,https://www.linkedin.com/company/cultureamp,2025-06-01T04:36:12,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Employee Engagement', 'Engagement Surveys', ...","['29 Stewart St, Level 2, Richmond, Victoria 3...",Culture Amp is the world’s leading employee ex...,False,Mid-Senior level,False,cultureamp,2025-05-02,Melbourne,VIC


In [16]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

32     FULL_TIME
33     FULL_TIME
34     FULL_TIME
35    CONTRACTOR
36    CONTRACTOR
Name: employment_type, dtype: object

In [17]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

32     1,001-5,000
33          51-200
34           11-50
35    5,001-10,000
36    5,001-10,000
Name: org_size, dtype: object

In [18]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1613031134,2025-05-02T05:33:16,2025-05-02T05:36:55.492225,Data Engineer,Data Engineer,NCS Group Australia,https://www.linkedin.com/company/ncs-group-aus...,2025-06-01T05:32:37,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,NCS in Australia is reinventing technology ser...,False,Mid-Senior level,True,ncs-group-australia,2025-05-02,Melbourne,VIC,FULL_TIME,"1,001-5,000"
1,1613035712,2025-05-02T05:21:43,2025-05-02T05:38:53.83865,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-01T05:21:43,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-02,Woden,None,CONTRACTOR,201-500


In [19]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1613031134,Data Engineer,Data Engineer,2025-05-02,Melbourne,VIC,FULL_TIME,NCS Group Australia,https://www.linkedin.com/company/ncs-group-aus...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, New South Wales",Privately Held,2000.0,"['IT Services', 'Technology Services', 'Artifi...","['Level 4, 10 Shelley Street, Sydney, New Sout...",NCS in Australia is reinventing technology ser...,False,Mid-Senior level,True,ncs-group-australia
1,1613035712,Full Stack Developer/ Senior Data Engineer,Data Engineer,2025-05-02,Woden,None,CONTRACTOR,Peoplebank,https://www.linkedin.com/company/peoplebank,https://au.linkedin.com/jobs/view/full-stack-d...,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
2,1612950483,AWS Data Engineer,Data Engineer,2025-05-02,Sydney,NSW,CONTRACTOR,BlueRose Technologies,https://www.linkedin.com/company/bluerosetechn...,https://au.linkedin.com/jobs/view/aws-data-eng...,...,"Bangalore, Karnataka",Privately Held,2010.0,"['PEGA', 'Telecom Billing', 'Siebel CRM', 'SAP...","['MS Square, 34/1-1, Langford Town, Bhemanna G...",BlueRose Technologies is a leading global prov...,False,Associate,True,bluerosetechnologies
3,1612910793,Staff Data Engineer - Data Intelligence - Real...,Data Engineer,2025-05-02,Sydney,NSW,FULL_TIME,Culture Amp,https://www.linkedin.com/company/cultureamp,https://au.linkedin.com/jobs/view/staff-data-e...,...,"Richmond, Victoria",Privately Held,2010.0,"['Employee Engagement', 'Engagement Surveys', ...","['29 Stewart St, Level 2, Richmond, Victoria 3...",Culture Amp is the world’s leading employee ex...,False,Mid-Senior level,False,cultureamp
4,1612905307,Staff Data Engineer - Data Intelligence - Real...,Data Engineer,2025-05-02,Melbourne,VIC,FULL_TIME,Culture Amp,https://www.linkedin.com/company/cultureamp,https://au.linkedin.com/jobs/view/staff-data-e...,...,"Richmond, Victoria",Privately Held,2010.0,"['Employee Engagement', 'Engagement Surveys', ...","['29 Stewart St, Level 2, Richmond, Victoria 3...",Culture Amp is the world’s leading employee ex...,False,Mid-Senior level,False,cultureamp


## Connect to  Snowflake database for raw data initial processing

In [20]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [21]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_10712/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(258, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [ ]:
##Deuplicate the data
df.drop(columns=['ID'], inplace=True)
df_dup= df.drop_duplicates(keep='last')
df_dup.shape

(258, 30)

In [17]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [18]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

17 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
12,1609347814,Senior Software Engineer - Enterprise Data Col...,Data Engineer,2025-05-01,Sydney,NSW,FULL_TIME,Nuix,https://www.linkedin.com/company/nuix,https://au.linkedin.com/jobs/view/senior-softw...,...,"Sydney, NSW",Public Company,2000.0,"['eDiscovery Tools', 'Insider Threats', 'Digit...","['Level 27, 1 Market St, Sydney, NSW 2000, AU'...",Nuix (www.nuix.com) creates innovative softwar...,False,Mid-Senior level,False,nuix
13,1611456228,Analyst | Workplace Integrity Data,Data Analyst,2025-05-01,Melbourne,VIC,FULL_TIME,Deloitte,https://www.linkedin.com/company/deloitte,https://au.linkedin.com/jobs/view/analyst-work...,...,Worldwide,Privately Held,1845.0,"['Audit', 'Consulting', 'Financial Advisory', ...","['Worldwide, Worldwide, OO', '30 Rockefeller P...",Deloitte drives progress. Our firms around the...,False,Entry level,False,deloitte
14,1611456187,Senior Analyst | Workplace Integrity Data,Data Analyst,2025-05-01,Melbourne,VIC,FULL_TIME,Deloitte,https://www.linkedin.com/company/deloitte,https://au.linkedin.com/jobs/view/senior-analy...,...,Worldwide,Privately Held,1845.0,"['Audit', 'Consulting', 'Financial Advisory', ...","['Worldwide, Worldwide, OO', '30 Rockefeller P...",Deloitte drives progress. Our firms around the...,False,Associate,False,deloitte
15,1611456174,Senior Analyst | Workplace Integrity Data,Data Analyst,2025-05-01,Sydney,NSW,FULL_TIME,Deloitte,https://www.linkedin.com/company/deloitte,https://au.linkedin.com/jobs/view/senior-analy...,...,Worldwide,Privately Held,1845.0,"['Audit', 'Consulting', 'Financial Advisory', ...","['Worldwide, Worldwide, OO', '30 Rockefeller P...",Deloitte drives progress. Our firms around the...,False,Associate,False,deloitte
16,1611456177,Analyst | Workplace Integrity Data,Data Analyst,2025-05-01,Sydney,NSW,FULL_TIME,Deloitte,https://www.linkedin.com/company/deloitte,https://au.linkedin.com/jobs/view/analyst-work...,...,Worldwide,Privately Held,1845.0,"['Audit', 'Consulting', 'Financial Advisory', ...","['Worldwide, Worldwide, OO', '30 Rockefeller P...",Deloitte drives progress. Our firms around the...,False,Entry level,False,deloitte


In [19]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()
def translate_text(text, target_language='en'):
    try:
        if not text:
            return 'NA'
        translated_text = translator.translate(text, dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

df_new_jobs['CITY'] = df_new_jobs['CITY'].apply(lambda x: translate_text(x, target_language='en'))

df_new_jobs['STATE'] = df_new_jobs['STATE'].apply(lambda x: translate_text(x, target_language='en'))

df_new_jobs['ORGANIZATION'] = df_new_jobs['ORGANIZATION'].apply(lambda x: translate_text(x, target_language='en'))

df_new_jobs['SENIORITY'] = df_new_jobs['SENIORITY'].apply(lambda x: translate_text(x, target_language='en'))

In [20]:

df_new_jobs['CITY'].unique()

array(['Melbourne', 'Woden', 'Sydney', 'Brisbane', 'Newcastle',
       'No Regrets'], dtype=object)

In [21]:
df_new_jobs['STATE'].unique()

array(['VIC', 'NA', 'NSW', 'QLD'], dtype=object)

In [22]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
